In [ ]:
! pip install ipyvizzu

In [ ]:
import pandas as pd
from ipyvizzu import Chart, Data, Config, Style

In [ ]:
nyphil_df = pd.read_csv(
    'nyphil_expansive_programming_data_by_season.csv'
    )

In [ ]:
# Create a dataframe with only new work information
new_works_df = nyphil_df[['season_year', 'decade', 'new_works', 'new_works_p']].assign(type ="New Works")

new_works_renamed = new_works_df.rename(columns={'new_works': 'total', 'new_works_p': 'percentage'})

In [ ]:
# Create a dataframe with only new composer information
new_composers_df = nyphil_df[['season_year', 'decade', 'new_composers', 'new_composers_p']].assign(type ="New Composers")

new_composers_renamed = new_composers_df.rename(columns={'new_composers': 'total', 'new_composers_p': 'percentage'})

new_composers_renamed['total'] = new_composers_renamed['total'].astype(int)

In [ ]:
# Merge composer and work dataframes
nyphil_vizzu_df = pd.concat([new_composers_renamed, new_works_renamed]).sort_values('season_year')

# Recast category columns as strings so ipyvizzu recognizes them as dimensions
nyphil_vizzu_df['season_year'] = nyphil_vizzu_df['season_year'].astype(str)
nyphil_vizzu_df['decade'] = nyphil_vizzu_df['decade'].astype(str)

# Convert percentage to number out of 100
nyphil_vizzu_df['percentage'] = nyphil_vizzu_df['percentage']*100

nyphil_vizzu_df.head(15)

In [ ]:
data = Data()
data.add_df(nyphil_vizzu_df)

style = Style(
    {
        "plot": {
            "yAxis": {
                "label": {
                    "fontSize": "1em",
                    "paddingRight": "1.2em",
                },
            },
            "xAxis": {
                "label": {
                    "angle": "2.75",
                    "fontSize": "1.1em",
                    "paddingRight": "0em",
                    "paddingTop": "1em",
                },
            "title": {"fontSize": "1.166667em"},
            },
            "marker": {"borderWidth": 1.5, "colorPalette": "#4171CDFF #F4941BFF"},
        },
        "logo": {"filter": "color(#FFFFFFFF)"},
        "legend": {"width": 150},
        "title": {"fontSize":"1.8em"}
    }
)

In [27]:
chart = Chart(width="850px", height="650px", display="manual")

chart.animate(data, style)

chart.feature("tooltip", True)

handler = """
let Year = parseFloat(event.detail.text);
if (Year % 10 !== 0 && Year > 1700) {
    event.preventDefault();
}
"""

x_reduce = chart.on("plot-axis-label-draw", handler)

chart.animate(
    Config({
        "x": "season_year",
        "y": ["total"],
        "geometry": "line",
        "color": 'type',
        "title": "Programming by Season: New Works and Composers",
        }),
)

chart.animate(
    Config({
        "y": "percentage",
        "title": "Proportion of Season: New Works and Composers"
    }),
    delay=6,
    duration=2
)

chart.animate(
    Config({
        "x": ["decade"],
        "y": "mean(percentage)",
        "title": "Average Proportion of Season by Decade: New Works and Composers"
        }),
    delay=6,
    duration=3
)

chart.animate(
    Config({
        "x": "season_year",
        "y": ["total"],
        "title": "Programming by Season: New Works and Composers"
        }),
    delay=8
)

chart.show()